In [22]:
#Install Packages
!# pip install faiss-cpu
# !pip install sentence-transformers

'#' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [24]:
df = pd.read_csv("../Data/sample_text.csv")
df.shape

(8, 2)

In [25]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Create source embeddings for the text column

In [26]:
from sentence_transformers import SentenceTransformer

In [27]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [28]:
vectors.shape

(8, 768)

In [29]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [30]:
import faiss

index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [31]:
index.add(vectors)

In [32]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000002A72A9A32D0> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [33]:
# search_query = "I want to buy a polo t-shirt"
search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [34]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [35]:
faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [37]:
distances, I = index.search(svec, k=2)
distances

array([[0.9273728, 1.1601744]], dtype=float32)

In [38]:
I

array([[6, 7]], dtype=int64)

In [39]:
I.tolist()

[[6, 7]]

In [40]:
row_indices = I.tolist()[0]
row_indices

[6, 7]

In [42]:
search_query

'looking for places to visit during the holidays'

In [43]:
df.loc[row_indices]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


You can see that the two results from the dataframe are similar to a search_query